In [1]:
from transformers import ViTFeatureExtractor

model_name_or_path = 'google/vit-large-patch32-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)
feature_extractor

c:\Users\aqila\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\aqila\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


ViTFeatureExtractor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTFeatureExtractor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [2]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['labels'] = example_batch['label']
    return inputs

In [3]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [4]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

C:\Users\aqila\AppData\Local\Temp\ipykernel_25736\2425197347.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [1]:
import os, shutil;
from PIL import Image
from sklearn.model_selection import train_test_split

directory_1 = "./ffhq_deepfake_detection/ffhq_faces"
directory_2 = "./ffhq_deepfake_detection/fake_ffhq"


images = []
labels = []


for root, dirs, files in os.walk(directory_1):
    for file in files:
        path = os.path.join(root,file)
        curr_image = Image.open(path)
        print(path)
        if curr_image.getbands() == ('R', 'G', 'B'):
          images.append(curr_image)
          labels.append(0)

for root, dirs, files in os.walk(directory_2):
    for file in files:
        path = os.path.join(root,file)
        curr_image = Image.open(path)
        if curr_image.getbands() == ('R', 'G', 'B'):
          images.append(curr_image)
          labels.append(1)



X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

./ffhq_deepfake_detection/ffhq_faces\images\00000.png
./ffhq_deepfake_detection/ffhq_faces\images\00001.png
./ffhq_deepfake_detection/ffhq_faces\images\00002.png
./ffhq_deepfake_detection/ffhq_faces\images\00003.png
./ffhq_deepfake_detection/ffhq_faces\images\00004.png
./ffhq_deepfake_detection/ffhq_faces\images\00005.png
./ffhq_deepfake_detection/ffhq_faces\images\00006.png
./ffhq_deepfake_detection/ffhq_faces\images\00007.png
./ffhq_deepfake_detection/ffhq_faces\images\00008.png
./ffhq_deepfake_detection/ffhq_faces\images\00009.png
./ffhq_deepfake_detection/ffhq_faces\images\00010.png
./ffhq_deepfake_detection/ffhq_faces\images\00011.png
./ffhq_deepfake_detection/ffhq_faces\images\00012.png
./ffhq_deepfake_detection/ffhq_faces\images\00013.png
./ffhq_deepfake_detection/ffhq_faces\images\00014.png
./ffhq_deepfake_detection/ffhq_faces\images\00015.png
./ffhq_deepfake_detection/ffhq_faces\images\00016.png
./ffhq_deepfake_detection/ffhq_faces\images\00017.png
./ffhq_deepfake_detection/ff

Traceback (most recent call last):
  File "C:\Users\aqila\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
  File "C:\Users\aqila\AppData\Local\Temp\ipykernel_39688\1281068203.py", line 16, in <module>
    curr_image = Image.open(path)
  File "c:\Users\aqila\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py", line 3227, in open
OSError: [Errno 24] Too many open files: './ffhq_deepfake_detection/ffhq_faces\\images\\08189.png'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\aqila\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 2105, in showtraceback
  File "C:\Users\aqila\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1428, in structured_traceback
  File "C:\Users\aqila\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1319, in structured_trac

In [ ]:
from datasets.dataset_dict import DatasetDict
from datasets import Dataset

d = {'train':Dataset.from_dict({'image':X_train,'label':y_train}),
     'validation':Dataset.from_dict({'image':X_val,'label':y_val}),
     'test':Dataset.from_dict({'image':X_test,'label':y_test})
     }

d = DatasetDict(d)


prepared_ds = d.with_transform(transform)

In [ ]:
prepared_ds['train']
type(prepared_ds)

In [ ]:
from transformers import ViTForImageClassification

labels = ['original', 'fake']

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

In [ ]:
from transformers import TrainingArguments, EarlyStoppingCallback


training_args = TrainingArguments(
  output_dir="./vit-base-beans2",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=5,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=feature_extractor,
    #callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]
)

In [ ]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

In [ ]:
metrics = trainer.evaluate(prepared_ds['test'])
trainer.log_metrics("test", metrics)
trainer.save_metrics("test", metrics)